<a href="https://colab.research.google.com/github/abdurrahmanbulut/song-popularity-prediction-ML/blob/main/classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats

import seaborn as sns
sns.set_style('darkgrid')

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

In [ ]:
track = pd.read_csv('./drive/MyDrive/colab/data/tracks.csv')

In [ ]:
track.drop(['name', 'id', 'artists', 'id_artists'], axis=1, inplace=True)

In [ ]:
track['release_date'] = np.where(track['release_date'].str.len() > 4, track['release_date'].str.slice(start=0, stop=4), track['release_date']).astype(str).astype(int)
before1922 = track[track['release_date'].astype(int) < 1922].index
track.drop(before1922, inplace=True)
track.reset_index(drop=True)

,popularity,duration_ms,explicit,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,6,126903,0,1922,0.645,0.4450,0,-13.338,1,0.4510,0.674,0.744000,0.1510,0.1270,104.851,3
1,0,98200,0,1922,0.695,0.2630,0,-22.136,1,0.9570,0.797,0.000000,0.1480,0.6550,102.009,1
2,0,181640,0,1922,0.434,0.1770,1,-21.180,1,0.0512,0.994,0.021800,0.2120,0.4570,130.418,5
3,0,176907,0,1922,0.321,0.0946,7,-27.961,1,0.0504,0.995,0.918000,0.1040,0.3970,169.980,3
4,0,163080,0,1922,0.402,0.1580,3,-16.900,0,0.0390,0.989,0.130000,0.3110,0.1960,103.220,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
586666,50,258267,0,2020,0.560,0.5180,0,-7.471,0,0.0292,0.785,0.000000,0.0648,0.2110,131.896,4
586667,72,153293,0,2020,0.765,0.6630,0,-5.223,1,0.0652,0.141,0.000297,0.0924,0.6860,150.091,4
586668,70,187601,0,2020,0.535,0.3140,7,-12.823,0,0.0408,0.895,0.000150,0.0874,0.0663,145.095,4
586669,58,142003,0,2021,0.696,0.6150,10,-6.212,1,0.0345,0.206,0.000003,0.3050,0.4380,90.029,4


In [ ]:
zeroPopularity = track[track['popularity'] == 0]
zeroPopularity = zeroPopularity[zeroPopularity['release_date'] < 1940].index
zeroPopularity
track.drop(zeroPopularity, inplace=True)
track.reset_index(drop=True)
track.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 571213 entries, 0 to 586671
Data columns (total 16 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   popularity        571213 non-null  int64  
 1   duration_ms       571213 non-null  int64  
 2   explicit          571213 non-null  int64  
 3   release_date      571213 non-null  int64  
 4   danceability      571213 non-null  float64
 5   energy            571213 non-null  float64
 6   key               571213 non-null  int64  
 7   loudness          571213 non-null  float64
 8   mode              571213 non-null  int64  
 9   speechiness       571213 non-null  float64
 10  acousticness      571213 non-null  float64
 11  instrumentalness  571213 non-null  float64
 12  liveness          571213 non-null  float64
 13  valence           571213 non-null  float64
 14  tempo             571213 non-null  float64
 15  time_signature    571213 non-null  int64  
dtypes: float64(9), int64

In [ ]:
track = track.reset_index()

In [ ]:
track

,index,popularity,duration_ms,explicit,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,0,6,126903,0,1922,0.645,0.4450,0,-13.338,1,0.4510,0.674,0.744000,0.1510,0.1270,104.851,3
1,13,1,186800,0,1922,0.703,0.2800,0,-15.390,1,0.1740,0.995,0.000068,0.1630,0.8970,127.531,4
2,26,4,180800,0,1922,0.420,0.2880,7,-14.005,1,0.0701,0.994,0.000216,0.1960,0.4000,139.575,3
3,149,41,177133,0,1923,0.614,0.0423,4,-12.033,1,0.0535,0.996,0.002930,0.1830,0.2110,89.822,4
4,150,38,161933,0,1923,0.831,0.2620,8,-12.386,1,0.0942,0.984,0.912000,0.2040,0.9010,104.606,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
571208,586667,50,258267,0,2020,0.560,0.5180,0,-7.471,0,0.0292,0.785,0.000000,0.0648,0.2110,131.896,4
571209,586668,72,153293,0,2020,0.765,0.6630,0,-5.223,1,0.0652,0.141,0.000297,0.0924,0.6860,150.091,4
571210,586669,70,187601,0,2020,0.535,0.3140,7,-12.823,0,0.0408,0.895,0.000150,0.0874,0.0663,145.095,4
571211,586670,58,142003,0,2021,0.696,0.6150,10,-6.212,1,0.0345,0.206,0.000003,0.3050,0.4380,90.029,4


In [ ]:
track['popularity'] = pd.qcut(track['popularity'], q=2, labels=[0, 1])

In [ ]:
track.popularity

0         0
1         0
2         0
3         1
4         1
         ..
571208    1
571209    1
571210    1
571211    1
571212    1
Name: popularity, Length: 571213, dtype: category
Categories (2, int64): [0 < 1]

In [ ]:
y = track.loc[:, 'popularity']
X = track.drop('popularity', axis=1)

In [ ]:
scaler = StandardScaler()

X = scaler.fit_transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=20)

In [ ]:
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

In [ ]:
log_model = LogisticRegression()
knn_model = KNeighborsClassifier()
dec_model = DecisionTreeClassifier()
mlp_model = MLPClassifier()
svm_model = SVC()
lgbm_model = LGBMClassifier()
xgboost_model = XGBClassifier()

In [ ]:
log_model.fit(X_train, y_train)
knn_model.fit(X_train, y_train)
dec_model.fit(X_train, y_train)
mlp_model.fit(X_train, y_train)
svm_model.fit(X_train, y_train)
lgbm_model.fit(X_train, y_train)
xgboost_model.fit(X_train, y_train)

In [ ]:
log_acc = log_model.score(X_test, y_test)
knn_acc = knn_model.score(X_test, y_test)
dec_acc = dec_model.score(X_test, y_test)
mlp_acc = mlp_model.score(X_test, y_test)
# svm_acc = svm_model.score(X_test, y_test)
# lgbm_acc = lgbm_model.score(X_test, y_test)
# xgboost_acc = xgboost_model.score(X_test, y_test)

In [ ]:
print("Logistic Regression Accuracy:", log_acc)
print("K-Nearest-Neighbors Accuracy:", knn_acc)
print("Decision Tree Accuracy:", dec_acc)
print("Multi-layer Perceptron Accuracy:", mlp_acc)
print("Support vector machine Accuracy:", svm_acc)
print("Light Gradient Boosting Machine Accuracy:", lgbm_acc)
print("eXtreme Gradient Boosting Accuracy:", xgboost_acc)

Logistic Regression Accuracy: 0.7425305198291356
K-Nearest-Neighbors Accuracy: 0.7734238229733199
Decision Tree Accuracy: 0.8387175836231647


In [ ]:
fig = px.bar(
    x=["Logistic Regression", "K-Nearest-Neighbors", "Decision Tree", "Neural Network", "Support Vector Machine", "Light Gradient Boosting Machine Accuracy", "eXtreme Gradient Boosting Accuracy:"],
    y=[log_acc, knn_acc, dec_acc, mlp_acc, svm_acc],
    color=["Logistic Regression", "K-Nearest-Neighbors", "Decision Tree", "Neural Network", "Support Vector Machine", "Light Gradient Boosting Machine Accuracy", "eXtreme Gradient Boosting Accuracy:"],
    labels={'x': "Model", 'y': "Accuracy"},
    title="Model Accuracy Comparison"
)

fig.show()

In [ ]:
track.popularity.value_counts()

0    293807
1    277406
Name: popularity, dtype: int64

In [ ]:
import plotly.express as px


In [ ]:
fig = px.bar(
    x=["Logistic Regression", "K-Nearest-Neighbors", "Decision Tree"],
    y=[log_acc, knn_acc, dec_acc],
    color=["Logistic Regression", "K-Nearest-Neighbors", "Decision Tree"],
    labels={'x': "Model", 'y': "Accuracy"},
    title="Model Accuracy Comparison"
)

fig.show()